# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.77'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
df2=pf.merge_fit_files(path=DayFolder)

## First, get the metadata for this day

In [4]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['Ne', 'FG', 'Cap','window'],
 sort=True)
diad_meta[0]

'03 C1.txt'

## Now get all the important information out of the metadata files

In [5]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
print(meta['filename'].iloc[0])

100%|██████████| 39/39 [00:00<00:00, 113.38it/s]

Done
03 C1.txt


## Then get a simple file name you can stitch with the spectra

In [6]:
help(pf.extracting_filenames_generic)

Help on function extracting_filenames_generic in module DiadFit.importing_data_files:

extracting_filenames_generic(*, names, prefix=False, str_prefix=None, suffix=False, CRR_filter=True, str_suffix=None, file_ext=None)
    Takes filenames from a panda series (e.g., a column of a dataframe of metadata), outputs a numpy array that is consistent with the same function for 
    spectra, to allow stitching of spectra and metadata. 
    
    
    Parameters
    -----------
    names: Pandas.Series 
        Series of sample names, e.g., from 'filename' column of metadata output
    
    prefix: bool
        if True, has a number before the file name
    
    str_prefix: str
        The string separating the prefix from the file name (e.g. if file is 01 test, str_prefix=" ")
    
    suffix: bool
        if True, has a number or name after the filename
    
    str_suffix: str
        The string separating the filename from the suffix
    
    file_ext: str
        The file extension, e.g., '

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=True, str_prefix=" ",
   file_ext='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'C1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=True, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_ext='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=pd.merge(df2, meta, on='name_for_matching').reset_index(drop=True)
df_combo.head()

,Unnamed: 0,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,0,33 C17,103.393934,0.002283,0.001650,1285.568348,0.001384,1660.537187,1285.568398,2886.759871,...,June,1.0,34.465,30.0,4.0,5.0,"['0h', '2m', '2s']",3:21:40 PM\n,55300.0,1325.004
1,0,77 C39,103.463625,0.005187,0.004116,1285.348698,0.003915,1473.256000,1285.348698,2747.398583,...,June,1.0,34.553,30.0,4.0,5.0,"['0h', '2m', '2s']",8:05:47 PM\n,72347.0,1325.004
2,0,03 C1_CRR_DiadFit,104.671247,0.001651,0.001224,1282.606952,0.001086,18686.480015,1282.607002,50297.640280,...,June,1.0,34.889,30.0,4.0,5.0,"['0h', '2m', '2s']",11:18:18 AM\n,40698.0,1325.004
3,0,05 C2_CRR_DiadFit,104.670539,0.001651,0.001208,1282.612061,0.001045,19512.179599,1282.612111,52525.898280,...,June,1.0,34.646,30.0,4.0,5.0,"['0h', '2m', '2s']",11:24:50 AM\n,41090.0,1325.004
4,0,07 C3_CRR_DiadFit,104.664331,0.001788,0.001318,1282.651066,0.001156,17925.398193,1282.651116,48477.149093,...,June,1.0,34.322,30.0,4.0,5.0,"['0h', '2m', '2s']",11:36:26 AM\n,41786.0,1325.004


## If you have secondary phases, now is the time to merge those in

In [17]:
df_combo_sec_phase=pf.merge_in_carb_SO2(
df_combo=df_combo, file1_name='Carb_Peak_fits.xlsx', file2_name='SO2_Peak_fits.xlsx')
df_combo_sec_phase['Name_for_Secondary_Phases_str'] = df_combo_sec_phase['Name_for_Secondary_Phases'].str.extract('(\d+)').astype(int)
df_sorted = df_combo_sec_phase.sort_values(by='Name_for_Secondary_Phases_str')

df_sorted.to_excel('FI_fitting_w_sec_phases_1220.xlsx', index=False)

In [18]:
df_sorted['Name_for_Secondary_Phases_str']

2      1
3      2
4      3
5      4
6      5
7      6
8      7
9      8
10     9
11    10
12    11
13    12
14    13
15    14
16    15
17    16
0     17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
28    28
29    29
30    30
31    31
32    32
33    33
34    34
35    35
36    36
37    37
38    38
1     39
Name: Name_for_Secondary_Phases_str, dtype: int32

## Now lets get the different Pkl files to see how they differ

In [19]:
Ne_corr_1447_1117=pf.calculate_Ne_corr_std_err_values(pickle_str='Neon_corr_model_1117_1147.pkl',
        new_x=df_sorted['sec since midnight'], CI=0.67)
Ne_corr_1447_1117.head()
Ne_corr_1447_1117['preferred_values'].to_clipboard(excel=True, index=False, header=False)

In [23]:
df_sorted['filename_x'].to_clipboard(excel=True, index=False, header=False)

In [24]:
Ne_corr_1447_1220=pf.calculate_Ne_corr_std_err_values(pickle_str='Neon_corr_model_1220_1147.pkl',
        new_x=df_sorted['sec since midnight'], CI=0.67)
Ne_corr_1447_1220.head()
Ne_corr_1447_1220['preferred_values'].to_clipboard(excel=True, index=False, header=False)

In [15]:
df_sorted

,Unnamed: 0,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
2,0,03 C1_CRR_DiadFit,104.671247,0.001651,0.001224,1282.606952,0.001086,18686.480015,1282.607002,50297.640280,...,June,1.0,34.889,30.0,4.0,5.0,"['0h', '2m', '2s']",11:18:18 AM\n,40698.0,1325.004
11,0,19 C10,104.293657,0.002524,0.001787,1283.523492,0.001324,2413.101156,1283.523542,7140.901666,...,June,1.0,34.647,30.0,4.0,5.0,"['0h', '2m', '1s']",1:15:50 PM\n,47750.0,1325.004
12,0,21 C11,104.004247,0.004229,0.003046,1284.182664,0.002523,2999.024997,1284.182814,8577.534757,...,June,1.0,34.617,30.0,4.0,5.0,"['0h', '2m', '1s']",1:36:24 PM\n,48984.0,1325.004
13,0,23 C12,103.688654,0.004235,0.003187,1284.889317,0.002888,2225.062561,1284.889367,5285.588949,...,June,1.0,34.691,30.0,4.0,5.0,"['0h', '2m', '2s']",2:03:05 PM\n,50585.0,1325.004
14,0,24 C13,103.697025,0.004272,0.003156,1284.878708,0.002783,2135.066854,1284.878758,5095.299329,...,June,1.0,34.687,30.0,4.0,5.0,"['0h', '2m', '1s']",2:05:34 PM\n,50734.0,1325.004
15,0,27 C14_CRR_DiadFit,103.577490,0.003373,0.002533,1285.111261,0.002291,2617.090169,1285.111311,5568.548543,...,June,1.0,34.672,45.0,4.0,5.0,"['0h', '3m', '1s']",2:49:17 PM\n,53357.0,1325.004
16,0,29 C15,103.514806,0.002938,0.002268,1285.259254,0.002112,1686.676345,1285.259304,3374.389056,...,June,1.0,34.556,30.0,4.0,5.0,"['0h', '2m', '2s']",3:05:28 PM\n,54328.0,1325.004
17,0,31 C16,103.429863,0.002216,0.001641,1285.484345,0.001453,1677.450146,1285.484395,2974.127042,...,June,1.0,34.485,30.0,4.0,5.0,"['0h', '2m', '2s']",3:14:52 PM\n,54892.0,1325.004
0,0,33 C17,103.393934,0.002283,0.001650,1285.568348,0.001384,1660.537187,1285.568398,2886.759871,...,June,1.0,34.465,30.0,4.0,5.0,"['0h', '2m', '2s']",3:21:40 PM\n,55300.0,1325.004
18,0,35 C18,104.438881,0.001965,0.001414,1283.187630,0.001165,8168.001075,1283.187680,23394.170162,...,June,1.0,34.428,30.0,4.0,5.0,"['0h', '2m', '2s']",3:56:28 PM\n,57388.0,1325.004


In [ ]:
df_sorted['Corrected_Splitting']

KeyError: 'Corrected_Splitting'